In [1]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
import bokeh
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
from bokeh.models import HoverTool, Legend, SingleIntervalTicker, LinearAxis
from bokeh.models import ColumnDataSource
from bokeh.palettes import Category10
import itertools
output_notebook()
def color_gen():
    yield from itertools.cycle(Category10[10])

Loading BokehJS ...

In [2]:
data = pd.read_csv('data.csv')

In [3]:
#data.head()

In [4]:
# K/D stats
kd = data[['name', 'dt', 'ranked_kill', 'ranked_death']]
kd = kd.copy()
def abc(x):
    print(x, type(x))
    return 0
kd['kd'] = kd.apply(lambda x: x['ranked_kill']/max(1, x['ranked_death']), axis=1)
kd['dt2'] = pd.to_datetime(kd['dt'])
kd['just_date'] = kd['dt2'].dt.date
#kd.head()

In [5]:
names = ['BisonBud', 'TurtleBud', 'SpicyJambalaya', 'Montek_', 'Goose-.-', 'BunnyBud', 'agtemd', 'Elzen.-']

In [25]:
hover = HoverTool(
    tooltips = [
        ("Name", "@name"),
        ("Date", "@dt"),
        ("K/D", "@kd"),
    ],
    formatters={
        'date_time': 'datetime'
    },
)

p = figure(plot_width=960, plot_height=600, x_axis_type="datetime", tools=[hover, 'pan', 'box_zoom', 'reset', 'save'], title='Overall K/D over time')
color = color_gen()
leg_items = []

for i in names:
    p_rows = kd[kd['name']==i]
    c = next(color)
    p.step(x='dt2', y='kd', color=c, line_width=1, alpha=0.4, source=p_rows)
    r = p.scatter(x='dt2', y='kd', color=c, line_width=0.5, source=p_rows)
    leg_items.append((i, [r]))

legend = Legend(items=leg_items, location=(0, 0))
p.add_layout(legend, 'right')
p.xaxis.axis_label = "Date Time"
p.yaxis.axis_label = "Kill/Death"
p.y_range.start = 0
p.yaxis[0].ticker.desired_num_ticks = 15
p.xaxis[0].ticker.desired_num_ticks = 20
show(p)

In [7]:
def day_diff(x):
    xmin = x.min()
    xmax = x.max()
    return xmax-xmin

date_pivot = kd[kd['name'].isin(names)]
date_pivot = date_pivot.sort_values(by=['name', 'dt2'])
date_pivot = date_pivot[['name', 'ranked_kill', 'ranked_death', 'just_date', 'dt2']]
groups = date_pivot.groupby('name')
def mydiff(x):
    return x[['ranked_kill', 'ranked_death']].diff()
dp = groups.apply(mydiff)#.reset_index(0, drop=True)
dp.columns = ['kill_diff', 'death_diff']
dp = dp.sort_index()
for i in dp:
    date_pivot[i] = dp[i]
date_pivot = date_pivot.reset_index()
#date_pivot[date_pivot['name'] == 'agtemd'].head()

In [8]:
date_pivot = date_pivot.pivot_table(index=['name', 'just_date'], aggfunc=sum, values=['kill_diff', 'death_diff'])
date_pivot['daily_kd'] = date_pivot.apply(lambda x: x['kill_diff'] / max(1, x['death_diff']), axis=1)
date_pivot = date_pivot.fillna(0)
#date_pivot.head(20)

In [9]:
def return_best(x):
    return x.idxmax()

date_pivot_name = date_pivot['daily_kd'].unstack(level=0)
date_pivot_name = date_pivot_name.fillna(0)
date_pivot_name['best'] = date_pivot_name.apply(lambda x: return_best(x), axis=1)
#date_pivot_name.head()

In [10]:
HTML('<b>Number of Days with Highest Daily K/D</b>' + date_pivot_name['best'].value_counts().to_frame().to_html())

,best
BisonBud,109
Goose-.-,59
TurtleBud,54
SpicyJambalaya,33
Montek_,13
agtemd,8
Elzen.-,5
BunnyBud,2


In [11]:
# Number of games played + time played per person
time_stats = data[data['name'].isin(names)]
time_stats = time_stats[['name', 'dt', 'ranked_won', 'ranked_lost', 'time_played']]
time_stats['total_games'] = time_stats['ranked_won'] + time_stats['ranked_lost']
#time_stats.head()

In [12]:
ts_pivot = time_stats.copy()
ts_pivot['dt2'] = pd.to_datetime(ts_pivot['dt'])
ts_pivot['mnth_yr'] = ts_pivot['dt2'].apply(lambda x: x.strftime('%Y-%m'))
ts_pivot = ts_pivot.pivot_table(index=['name', 'mnth_yr'], aggfunc=max, values=['total_games', 'time_played'])
ts_pivot = ts_pivot.unstack('name')
ts_pivot = ts_pivot.fillna(method='ffill').fillna(0)
#ts_pivot.head(30)

In [13]:
ts_diff = ts_pivot.diff()
#ts_diff.head()

In [14]:
final_tt = ts_diff.stack().pivot_table(index='name', columns='mnth_yr', values='time_played')
final_tt = final_tt / 3600
final_tt.loc['Montek_', '2019-03'] = 0
#HTML(final_tt.to_html())

In [15]:
tt_data = pd.melt(final_tt.reset_index(0), var_name='month', id_vars='name')
#tt_data.head()

In [26]:
hover = HoverTool(
    tooltips = [
        ("Name", "@name"),
        ("Month", "@month"),
        ("Time", "@value hours"),
    ]
)

p = figure(plot_width=960, plot_height=600, tools=[hover, 'reset', 'save', 'pan', 'box_zoom'], 
           title='Play Time (Hours)', x_range=tt_data['month'].unique())
color = color_gen()
leg_items = []

dd = -0.5

for i in names:
    p_rows = tt_data[tt_data['name']==i]
    c = next(color)
    r = p.line(x='month', y='value', color=c, line_width=1, source=p_rows)
    p.scatter(x='month', y='value', color=c, line_width=1, source=p_rows)
    #r = p.line(x=[(i,dd) for i in p_rows['month']], top=p_rows['value'], color=c, width=0.1)
    leg_items.append((i, [r]))
    dd += 0.1

legend = Legend(items=leg_items, location=(0, 0))
p.add_layout(legend, 'right')
p.yaxis.axis_label = "Time (Hours)"
p.xaxis.axis_label = "Month"
#p.y_range.start = 0
#p.yaxis[0].ticker.desired_num_ticks = 15
#p.xaxis[0].ticker.desired_num_ticks = 20
show(p)

In [17]:
OPERATOR_LIST = [
    ('2:1', 'Smoke', 'smoke', 'def'),
    ('2:2', 'Castle', 'castle', 'def'),
    ('2:3', 'Doc', 'doc', 'def'),
    ('2:4', 'Glaz', 'glaz', 'atk'),
    ('2:5', 'Blitz', 'blitz', 'atk'),
    ('2:6', 'Buck', 'buck', 'atk'),
    ('2:7', 'Blackbeard', 'bb', 'atk'),
    ('2:8', 'Capitao', 'capitao', 'atk'),
    ('2:9', 'Hibana', 'hibana', 'atk'),
    ('2:10', 'Maverick', 'maverick', 'atk'),
    ('2:11', 'Nomad', 'nomad', 'atk'),
    ('2:12', 'Mozzie', 'mozzie', 'def'),
    ('2:A', 'Jackal', 'jackal', 'atk'),
    ('2:B', 'Ying', 'ying', 'atk'),
    ('2:C', 'Ela', 'ela', 'def'),
    ('2:D', 'Dokkaebi', 'dokkaebi', 'atk'),
    ('2:13', 'Nokk', 'nokk', 'atk'),
    ('2:14', 'Warden', 'warden', 'def'),
    ('2:F', 'Maestro', 'maestro', 'def'),
    ('3:1', 'Mute', 'mute', 'def'),
    ('3:2', 'Ash', 'ash', 'atk'),
    ('3:3', 'Rook', 'rook', 'def'),
    ('3:4', 'Fuze', 'fuze', 'atk'),
    ('3:5', 'IQ', 'iq', 'atk'),
    ('3:6', 'Frost', 'frost', 'def'),
    ('3:7', 'Valkyrie', 'valkyrie', 'def'),
    ('3:8', 'Caveira', 'cav', 'def'),
    ('3:9', 'Echo', 'echo', 'def'),
    ('3:10', 'Clash', 'clash', 'def'),
    ('3:11', 'Kaid', 'kaid', 'def'),
    ('3:12', 'Gridlock', 'gridlock', 'atk'),
    ('3:A', 'Mira', 'mira', 'def'),
    ('3:B', 'Lesion', 'lesion', 'def'),
    ('3:C', 'Zofia', 'zofia', 'atk'),
    ('3:D', 'Vigil', 'vigil', 'def'),
    ('3:E', 'Lion', 'lion', 'atk'),
    ('3:F', 'Alibi', 'alibi', 'def'),
    ('4:1', 'Sledge', 'sledge', 'atk'),
    ('4:2', 'Pulse', 'pulse', 'def'),
    ('4:3', 'Twitch', 'twitch', 'atk'),
    ('4:4', 'Kapkan', 'kapkan', 'def'),
    ('4:5', 'Jager', 'jager', 'def'),
    ('4:E', 'Finka', 'finka', 'atk'),
    ('5:1', 'Thatcher', 'thatcher', 'atk'),
    ('5:2', 'Thermite', 'thermite', 'atk'),
    ('5:3', 'Montagne', 'montagne', 'atk'),
    ('5:4', 'Tachanka', 'tach', 'def'),
    ('5:5', 'Bandit', 'bandit', 'def')
    ]

In [18]:
op_df = pd.DataFrame([], columns=['Operator', 'Atk/Def', 'K/D Best Player', 'K/D', 'Player Played Most', 'Hours'])
name_filtered = data[data['name'].isin(names)]
idx = name_filtered.groupby(['name'])['record_id'].transform(max) == name_filtered['record_id']
last_records = name_filtered[idx]

sorted_op_list = sorted(OPERATOR_LIST, key=lambda x: (x[3], x[1]))

for i in sorted_op_list:
    vals = {'Operator': i[1]}
    op = i[2]
    opk = op + '_k'
    opd = op + '_d'
    op_filtered = last_records[['name', 'dt'] + [i[2] + j for j in ['_tp', '_rw', '_rl', '_k', '_d']]]
    op_filtered['kd'] = op_filtered[opk] / op_filtered[opd]
    idmax = op_filtered['kd'].idxmax()
    vals['K/D Best Player'] = op_filtered.loc[idmax, 'name']
    vals['K/D'] = op_filtered.loc[idmax, 'kd']
    idmax = op_filtered[op + '_tp'].idxmax()
    vals['Player Played Most'] = op_filtered.loc[idmax, 'name']
    vals['Hours'] = op_filtered.loc[idmax, op + '_tp'] / 3600
    vals['Atk/Def'] = i[3]
    
    op_df = op_df.append(vals, ignore_index=True)
    


C:\Python64\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [19]:
HTML(op_df[op_df['Atk/Def']=='atk'].to_html())

,Operator,Atk/Def,K/D Best Player,K/D,Player Played Most,Hours
0,Ash,atk,BisonBud,1.756923,Goose-.-,63.780833
1,Blackbeard,atk,BunnyBud,1.769231,SpicyJambalaya,64.769167
2,Blitz,atk,BisonBud,0.857143,TurtleBud,43.332778
3,Buck,atk,BisonBud,1.489157,Goose-.-,40.191944
4,Capitao,atk,BisonBud,1.495575,BisonBud,27.802222
5,Dokkaebi,atk,BisonBud,1.710000,Montek_,27.061389
6,Finka,atk,TurtleBud,1.551724,Montek_,13.279167
7,Fuze,atk,BisonBud,1.547771,Goose-.-,16.243611
8,Glaz,atk,BisonBud,1.865979,TurtleBud,19.403611
9,Gridlock,atk,BisonBud,1.625000,TurtleBud,13.919444


In [20]:
HTML(op_df[op_df['Atk/Def']=='def'].to_html())

,Operator,Atk/Def,K/D Best Player,K/D,Player Played Most,Hours
24,Alibi,def,BisonBud,1.887755,BisonBud,8.719167
25,Bandit,def,BisonBud,1.737624,Elzen.-,73.785833
26,Castle,def,BisonBud,1.421053,agtemd,18.765000
27,Caveira,def,BisonBud,1.580645,BisonBud,18.641944
28,Clash,def,agtemd,1.000000,TurtleBud,1.715000
29,Doc,def,BisonBud,1.923505,BisonBud,70.066111
30,Echo,def,BisonBud,1.504854,Goose-.-,23.334444
31,Ela,def,BisonBud,1.997567,TurtleBud,42.976944
32,Frost,def,BisonBud,1.804348,SpicyJambalaya,23.265278
33,Jager,def,BisonBud,1.742537,Goose-.-,89.432222
